In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from deep_hedging import (
    Frequency,
    ConstantRateCurve,
    ZeroCouponBond,
    FixedCouponBond,
)
from deep_hedging.utils.fixing_dates import get_annual_indices

RANDOM_SEED = 12

In [3]:
RF_RATE = 0.15
Z_SPREAD = 0.05
CONVENIENCE_YIELD = 0.02

TIME_TILL_MATURITY = 3.0
FREQUENCY = Frequency.ANNUALLY

Equations:
1) $x_1 + x_2 + x_3 = 1$
2) $x_2 \cdot (1 + rf + z) - cpn \cdot (1 + rf - cy) = 0$
3) $x_3 \cdot (1 + rf + z)^2 - cpn \cdot (1 + rf - cy)^2 = 0$
4) $x_1 \cdot (1 + rf + z)^3 - (1 + cpn) \cdot (1 + rf - cy)^3 = 0$

Equations:
1) $x_1 + x_2 + x_3 = 1$
2) $x_2 \cdot (1 + rf + z) - cpn \cdot (1 + rf - cy) = 0$
3) $x_3 \cdot (1 + rf + z)^2 - cpn \cdot (1 + rf - cy)^2 = 0$
4) $x_1 \cdot (1 + rf + z)^3 - cpn \cdot (1 + rf - cy)^3 = (1 + rf - cy)^3$

In [4]:
k = np.array(
    [
        [1, 1, 1, 0],
        [0, (1 + RF_RATE + Z_SPREAD), 0, -(1 + RF_RATE - CONVENIENCE_YIELD)],
        [
            0,
            0,
            (1 + RF_RATE + Z_SPREAD) ** 2,
            -((1 + RF_RATE - CONVENIENCE_YIELD) ** 2),
        ],
        [
            (1 + RF_RATE + Z_SPREAD) ** 3,
            0,
            0,
            -((1 + RF_RATE - CONVENIENCE_YIELD) ** 3),
        ],
    ]
)
y = np.array([1, 0, 0, (1 + RF_RATE - CONVENIENCE_YIELD) ** 3])

In [5]:
x = np.linalg.solve(k, y)
x

array([0.88673611, 0.05833333, 0.05493056, 0.0619469 ])

In [6]:
print(f"{x[-1] * 100:.4f}%")

6.1947%


## General Solution.

In [7]:
factors = get_annual_indices(
    TIME_TILL_MATURITY,
    FREQUENCY,
)
factors

array([1., 2., 3.])

In [8]:
k_0 = np.array([[1.0] * len(factors) + [0.0]])
k_other = []
for i, factor in enumerate(factors):
    line = [0.0] * (len(factors) + 1)
    line[i] = (1 + RF_RATE + Z_SPREAD) ** factor
    line[-1] = -((1 + RF_RATE - CONVENIENCE_YIELD) ** factor)
    k_other.append(line)

k_other = np.array(k_other)
k_general = np.concatenate([k_0, k_other], axis=0)
k_general

array([[ 1.      ,  1.      ,  1.      ,  0.      ],
       [ 1.2     ,  0.      ,  0.      , -1.13    ],
       [ 0.      ,  1.44    ,  0.      , -1.2769  ],
       [ 0.      ,  0.      ,  1.728   , -1.442897]])

In [9]:
k

array([[ 1.      ,  1.      ,  1.      ,  0.      ],
       [ 0.      ,  1.2     ,  0.      , -1.13    ],
       [ 0.      ,  0.      ,  1.44    , -1.2769  ],
       [ 1.728   ,  0.      ,  0.      , -1.442897]])

In [10]:
y_general = np.array(
    [1.0]
    + [0.0] * (len(factors) - 1)
    + [(1 + RF_RATE - CONVENIENCE_YIELD) ** factors[-1]]
)
y_general

array([1.      , 0.      , 0.      , 1.442897])

In [11]:
y

array([1.      , 0.      , 0.      , 1.442897])